# Assessment 1 Parsing Data And Text Preprocessing
## Task 2: Text Pre-Processing
*   Name: Jingyi Chen
*   ID: 31314295

### 1.0 Initializing the Environment
> Load the google drive path to read the data source;

> Import the packages needed for programming. 

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import the packages needed for programming. 
import re
import os
import pandas as pd
from math import ceil
import nltk
from nltk.tokenize import RegexpTokenizer 
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### 2.0 Data Cleaning
This part is mainly to clean the data as a whole. Because there are many pages of excel data, and the data on each page may be different. We need to clean the NA value and outliers of the data, and then merge the data together to facilitate our subsequent data operations and calculations.

In [ ]:
# read the excel data from google drive
directory = "/content/drive/Shareddrives/FIT5196-s1-2022/A1/Task2/input_data/31314295.xlsx"
excel_data = pd.ExcelFile(directory)
excel_data

There are 20 pages data in this excel file.

In [ ]:
# check the excel page number
excel_data.sheet_names

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19']

#### 2.1 Cleaning NA Data and Column Name Data 
This function **data_clean** is to clean a sheet of Excel data (according to week1- Week3 lessons). First load excel data, then delete all NA columns and all NA rows. As the operation of deleting NA causes the index of row and column number to be discontinuous, it is necessary to correct the number of rows and columns, and finally change the column names to 'reviewText', 'summary', 'reviewTime'.

In [ ]:
def data_clean(sheet_no):
  df = excel_data.parse(sheet_no)
  df = df.dropna(axis= 1, how = 'all')
  df = df.dropna(axis= 0, how = 'all')
  df.index = range(len(df.index))
  df.columns = list(range(len(df.columns))) 
  df.columns = ['reviewText', 'summary', 'reviewTime']
  return df

First, excel data with sheet number 0 is cleaned, and then data on each page is cleaned through the **for loop**. Also in the loop, two piece of data in  Dataframe formats are **merged** according to the same column name through **concat** function. Because the **column names** in each sheet are read as data, some sheets produce a row of extra data. Data that removes column names from row data after concatenation.

In [ ]:
df = data_clean('0')

for sheet in excel_data.sheet_names:
  if(sheet != '0'):
      df_sheet = data_clean(sheet)
      df = pd.concat([df_sheet, df], axis = 0, ignore_index=True)
      df = df[~((df['reviewText'] == 'reviewText') & (df['summary'] == 'summary') & (df['reviewTime'] == 'reviewTime'))]
      

#### 2.2 Data validation after cleaning 
After data cleaning, we ended up with **10000 rows** and **3 columns** in Dataframe format. And the **column name** of the data is correct. Check the first five lines to **make sure the data is properly cleaned**.

In [ ]:
# check the row number and col number
df.shape

(10000, 3)

In [ ]:
# check the cols name
df.columns

Index(['reviewText', 'summary', 'reviewTime'], dtype='object')

In [ ]:
# check the data of the first 5 rows 
df.head()

,reviewText,summary,reviewTime
1,The snobs that think Ivan Reitman's film isn't...,"Lighten Up, Folks...","12 6, 2010"
2,I was really surprised by this book on how it ...,5 Stars are not enough it should be 10+.,"07 7, 2014"
3,The exact replacement for a APC RBC109 Battery...,This is not a exact replacement for a APC Batt...,"01 18, 2013"
4,I bought this to clean the things on my desk i...,It really works,"09 21, 2012"
5,"I highly recommend ""V"" to anyone, who enjoys s...","Entertaining, suspenseful, and exciting!","01 20, 2006"


### 3.0 Text Preprocessing
 In this part, we mainly divide the sentences, and then do some cleaning work on the words, including removing the stop words, Rare tokens (with the threshold set to less than 10 days (i.e. 10 unique dates)), the words of the threshold to more than ceil(Number_of_days / 2). And modifying the words by stemming operation. Then finally completing the word frequency statistics and writing the data into the file.

 The following **seven steps** describe the overall word processing process in detail.

####  First Step: 
We need to **convert the data format** to str, because the text data may contain some numeric data(Int, float), affecting the subsequent segmentation.

In [ ]:
df = df.astype(str)

Group **‘reviewTime’** fields using the built-in dataframe method group, find **‘reviewText’** of the same date, and then use **‘reviewTime’** as the key and **‘reviewText’** as the value of the array type. Then we get a dictionary type data, which can be caculated the number of words.

In [ ]:
# First Step: Group **‘reviewTime’** fields, find **‘reviewText’** of the same date
dict_word = dict()
groups_= df.groupby('reviewTime').groups
for key in groups_:
  if key not in dict_word:
    dict_word[key] = []
  for index in groups_[key]:
    dict_word[key].append(df['reviewText'][index])

####  Second Step: 

According the regular expression **'[a-zA-Z]+(?:[-'][a-zA-Z]+)?'** to split the sentence to words by **re model**. Then lower case words by **lower function** and eliminate duplicate words by **set** type transform to prevent multiple words in a day. 

In [ ]:
# Second Step: split the sentence to words and eliminate duplicate words
reg_tokens = re.compile(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
uniq_tokens = []

# split the sentence to words
for key in dict_word:
    word_temps = []
    words_string = " ".join(dict_word[key])
    temp = re.findall(reg_tokens, words_string)

    # lower words
    for word in temp:
        word_temps.append(word.lower())
    uniq_tokens += list(set(word_temps))

####  Third Step: 

Load the stop words dataset and Python's english stop words package to ensure removing the stop words. Put these two kinds of stop words list in the same list and drop the duplicate stop words. Finally removing the stop words in words bag that we have got.

In [ ]:
# Third Step: removing the stop words

# Load the basic stopwords list
stopwords_list = stopwords.words('english')

# Load the stopwords file
directory_stopwords = "/content/drive/Shareddrives/FIT5196-s1-2022/A1/Task2/stopwords_en.txt"
with open(directory_stopwords,'r') as infile:
    stopwords = infile.read().splitlines()

# concat two stopwords list
for stopword in stopwords:
  stopwords_list.append(stopword)
stopwords_set = set(stopwords_list) 

# removing stopwords
match_tokens = [] # no-stop words
for each in uniq_tokens:
    if each not in stopwords_set:
        match_tokens.append(each)

####  Fourth Step: 

Now that we have the word bag under each date (no duplicate words), we are going to do the word count, using **FreqDist** function to calculate the word frequency. Then Removing the words that less than 10 days (i.e. 10 unique dates)) and more than ceil(Number_of_days / 2).




In [ ]:
# Fourth Step: removing the words that less than 10 days (i.e. 10 unique dates)) and more than ceil(Number_of_days / 2). 

# caculating the frequencies of unique words at each date
freq_uniq_tokens = []
frequency_tokens = FreqDist(match_tokens)

# removing the words that less than 10 days and more than ceil(Number_of_days / 2)
for token in frequency_tokens:
    if frequency_tokens[token] >=10 and frequency_tokens[token] <= ceil(len(dict_word.keys())/2):
        freq_uniq_tokens.append(token)

####  Fifth Step: 
  
  Using the **Porter Stemmer** to access the stem of word and removing the tokens that less than 3 by **for loop**.

In [ ]:
# Fifth step: Using the Porter Stemmer to access the stem of word
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stem_token = []
for token in freq_uniq_tokens:
    stem_token.append(stemmer.stem(token))

# remove the tokens that less than 3    
vocab_m3 = []
for token in stem_token:
    if len(token) > 2:
      vocab_m3.append(token)

####  Sixth step: 

First of all, all the words are obtained through the for loop and word segmentation. After converting the words to lowercase, use the built-in method nbest under the **nltk package** to find out the 200 most meaningful phrases. These words are spliced and put into the list, and then **merged** with the previous words that have been cleared.


In [ ]:
# Sixth step: Get the 200 meaningful bigrams by pmi measure
bigrams = nltk.collocations.BigramAssocMeasures()

token_list = [] # all tokens

# split the sentence to words
for date in dict_word:
    words_string = " ".join(dict_word[date])
    token_list += re.findall(reg_tokens, words_string)

# lower words
tokens_list = []
for token in token_list:
    tokens_list.append(token.lower())

# nltk package
finder = nltk.collocations.BigramCollocationFinder.from_words(tokens_list)
meaning = finder.nbest(bigrams.pmi, 200)

meaning_vocab = []
for token in meaning:
    word = token[0] + '_' + token[1]
    meaning_vocab.append(word)
# Get all the vocab 
all_vocab = meaning_vocab + vocab_m3

####  Seventh step: 

Sort these words in ascending alphabetical order by **sort funtion**, and then **zipper** the words that have been counted and the words we really need after cleaning. And finally write these data to the file by using the **open** function.

In [ ]:
# sort the word by first letter
all_vocab.sort()

num_list = []
for i in range(len(all_vocab)):
    num_list.append(i)

# zip the two word list
all_vocab_dict = dict(zip(all_vocab, num_list))

# write the word to file
res = ''
for key, value in all_vocab_dict.items():
    res += key + ':' + str(value) +'\n'

with open('/content/drive/MyDrive/2020 August - Master of Data Science/task2_sample_vocab.txt','w') as f:
    f.write(res)

##4.0 Creating the sparse matrix using countvectorizer.
The main purpose of this part is to generate a matrix of words on different dates. Mainly through the method of **CountVectorizer** to achieve. Generate word bag, word frequency, and final word matrix, respectively.. 

Initialize CountVectorizer:

 CountVectorizer function description: Create the word bag data structure
 
 Parameters:

> Transfor the word to Lower case

> Regular expression word segmentation

> Drop stop words

> According to unigram and bigram






  

In [ ]:
vector = CountVectorizer(lowercase=True, token_pattern = "[a-zA-Z]+(?:[-'][a-zA-Z]+)?",  stop_words=stopwords, ngram_range=(1, 2), analyzer='word')

Use a two-layer for loop to produce the result we want, generating a sparse matrix on demand. Loop through the dictionary with the date as the key generated before, iterate over each sentence in the value list, modify the format of the date, and then do the inner loop to convert the word into the number after the feature for concatenation, and finally write to the file.

In [ ]:
res = ""
for key_date in dict_word: # dict_word structure: {date:[text,text]}
    words_string = " ".join(dict_word[key_date])

    # Statistical results of word frequency: (0,2) 1 
    # Used to process data, expressed in n-gram form
    nums = vector.fit_transform([words_string]).toarray()[0] 

    name = vector.get_feature_names() # vocabulary list (a:0,b:1)

    # date transform
    trans_date = ""
    key_date = re.split('\s', key_date)

    key_date[1] = key_date[1][:-1]
    if len(key_date[0]) == 1:
        key_date[0] = '0' + key_date[0]
    if len(key_date[1]) == 1:
        key_date[1] = '0' + key_date[1]
        
    trans_date = '/'.join(key_date) 
    
    res += trans_date + ','

    # Concatenate word numbers and word frequencies in the dictionary
    daily_dic = {}
    for i in range(len(name)):
        daily_dic[name[i]] = nums[i]
       
    for i in range(len(name)):
        each = name[i]
        num = nums[i]
        
        if daily_dic[each] <= 0:
            continue
        words = each.split()
        if len(words) == 2:
            word = '_'.join(words)
        else:
            word = stemmer.stem(each)

        ## check the word in the word bag
        if word in all_vocab_dict:
            res += str(all_vocab_dict[word]) + ":" + str(num) + ','
    res = res[:-1] + '\n'

## write the data to the file
with open('/content/drive/MyDrive/2020 August - Master of Data Science/output_countVec.txt', 'w') as f:
    f.write(res)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
